In [1]:
using constants
using types
using Optim
using PyPlot
using data
include("DOSfunctions.jl")
include("userfunctions.jl")
include("propertyfuncs.jl")

LoadError: LoadError: syntax: invalid type signature
while loading /home/abdullah/working_directory/jupyter_wd/julia/scienctific/BTE/types.jl, in expression starting on line 8

In [2]:
Ts=collect(150.01:50.0:1200.01)
#Ts=collect(663.8:0.01:664.0)#[663.0,663.1]
numofn=[-1.5e20,-7.3e19,-2.0e19,-0.22e19]#,-0.2e20]#,-1.75e20,-3.0e19,-0.9e20]#Sample 1 -3.0e19, Sample 2 -1.25e20,
#Sample 3 -1.75e20, Sample Sb 0.75% -1.03e20, #
numofnn=[1e17,1e17,1e17,1e17]#,1e17]
xs=[0.3,0.3,0.3,0.262]#[0.287,0.3,0.262,0.332]#collect(0.0:0.01:1.0)

var0=[Ts[1],xs[1]]
Eoff=5.0
Ev(var)=Eoff
Ec(var)=Ev(var)+EgSiGe(var[1],var[2])#var[1]:T var[2]:x
alphanp=0.0#1/EgSiGe(Ts[1],xs[1])


effMassV(var)=-1.2*me
effMassC(var)=0.322*me#1.4/6^(2/3)*me#0.321me#0.424*me#0.424*me#1.4/6^(2/3)*me

bandC=parBandTx(effMassC(var0),alphanp,Ec(var0),[Ec],[effMassC],var0)
bandV=parBandTx(effMassV(var0),0.0,Ev(var0),[Ev],[effMassV],var0)
#bndstTx=BandStrucTx([bandC,bandC,bandC,bandC,bandC,bandC],var0)
bndstTx=BandStrucTx([bandC,bandC,bandC,bandC,bandC,bandC,bandV],var0)#,bandC,bandC,bandC,bandC,bandC,bandV],var0)
#var array: var[1]: effMass , var[2]:T, var[3]:E, var[4]:x, var[5]:Ef, 
#var[6]:band, var[7]:NII var[8]:bndst var[9]:NNI var[10]:bandV
function Deftemp(band::parBandTx)
    Da=7.0
    Dv=4.0
    if band==bandC
        return [Da,Dv]#10.5#15.0
    end
    if band==bandV        
        return [Dv,Da]#7.0
    end
end
ksi=21.0
tauAC(var)=tauAC_func(ClSiGe(var[4],var[2]),Deftemp(var[6]),var[2],var[1],var[3],var[6])#DefP(var[1])
#tauAC(var)=tauAC_func(ClSiGe(var[4],var[2]),Deftemp(var[6])[1],Deftemp(var[6])[2],var[2],var[1],var[3],var[6])#DefP(var[1])
#tauAC(var)=tauAC_func(ClSiGe(var[4],var[2]),Deftemp(var[6])[1],var[2],var[1],var[3],var[6])#DefP(var[1])
tauII2(var)=tauII2_func(ksi*eps0,eps0,var[6],var[5],var[2],var[7],var[1],var[3],var[8])
tauNI(var)=tauNI_func(ksi*eps0,var[1],var[9])
vartau0=[bandC.effMass,Ts[1],Ec(var0),xs[1],Ec(var0),bandC,numofn[1]*1e6,bndstTx,numofnn[1]*1e6,bandV]
#vartau0=[bandC.effMass,Ts[1],1.0,xs[1],1.5,bandC,numofn[1]*1e6,bndstTx,numofnn[1]*1e6]
tauTOTTx=tau_electron_B([tauAC,tauII2],vartau0)#tauNI,,tauII2,tauAC,
xmax=3.0
#T_EF_Dict=Dict()
#println("-3")

LoadError: UndefVarError: parBandTx not defined

In [3]:
@time (Efs,sigmas,seebecks,kes,kees,kehs,kbis)=electronicPropsiso(bndstTx,xs,Ts,numofn,tauTOTTx,xmax)

Efsr=Array{Float64,3}(length(Ts),length(numofn),length(xs))
for (i,iT) in enumerate(Ts)
    for (k,kx) in enumerate(xs) 
        for (j,jn) in enumerate(numofn)
            Efsr[i,j,k]=Efs[i,j,k]-Ec([iT,kx])
        end
    end
end
#println("-2")
T_EF_Dict=Dict()
for (Ti,Tx) in enumerate(Ts)
    T_EF_Dict[Tx]=(Efs[Ti])    
end
T_EF_Dict_Array=Array{Dict{Float64,Float64},2}(length(numofn),length(xs))
Dict_temp=Dict()
Efsr=Array{Float64,3}(length(Ts),length(numofn),length(xs))
for (j,jn) in enumerate(numofn)
    for (k,kx) in enumerate(xs) 
        for (i,iT) in enumerate(Ts)
            Efsr[i,j,k]=Efs[i,j,k]-Ec([iT,kx])
            Dict_temp[iT]=Efsr[i,j,k]
            T_EF_Dict_Array[j,k]=Dict_temp 
        end
    end
end

LoadError: UndefVarError: electronicPropsiso not defined

In [4]:
#println("-1")
#############Lattice Thermal Conductivity Parameters###############
thetaSiL=586.0 # K 
thetaSiT=240.0 # K
vSiL=8430.0    # m/s
vSiT=5840.0    # m/s
vGeL=4920.0    # m/s
vGeT=3540.0    # m/s
gammaL=0.32
gammaT=0.7
MSi=4.6637e-26 # kg
MGe=1.206e-25  # kg
VSi=2.0024e-29 #m3
VGe=2.2636e-29 #m3
md=effMassC([Ts[1],xs[1]])*6^(2/3)#1.4*me#;
Eep=2.8*q
xsp=xs[1]
Tp=300.0
p=1.0
D=20e-9
NP=0.0
a=0.44
alpha=0.5#specularity factor
d=97.0e-9#

#println("1")

#vartauPH0=[[0.02],Ts[1],gammaL,gammaT,xsp,thetaCLx,thetaCTx,thetaCTx,MSi,MGe,VSi,a,T_EF_Dict,0.0]
vartauPH0=[ [0.02],  Ts[1],   gammaL,  gammaT, xsp,
    thetaSiL, thetaSiT, thetaSiT, MSi,   MGe, 
    VSi, a, T_EF_Dict_Array, Eep, alpha, 
    d, thetaSiL, thetaSiT, vSiL, vSiT, 
    vGeL, vGeT, p, NP,Tsindex=1,
    numberofnindex=1,xsindex=1]
thetaCLx=thetaCL(vartauPH0[12],vartauPH0[5],vartauPH0[17],vartauPH0[19],vartauPH0[11])
thetaCTx=thetaCT(vartauPH0[12],vartauPH0[5],vartauPH0[18],vartauPH0[19],vartauPH0[11])
#println("2")
tauPHNLf(var)=tauPH_NL(var[3],VSiGe(var[5]),var[1],var[2],MSiGe(var[5]),vSiGeL_PH(var[5]))
tauPHNTf(var)=tauPH_NT(var[4],VSiGe(var[5]),var[1],var[2],MSiGe(var[5]),vSiGeT_PH(var[5]))

tauPHULf(var)=tauPH_U(var[3],var[1],var[2],MSiGe(var[5]),vSiGeL_PH(var[5]),var[17])
tauPHUTf(var)=tauPH_U(var[4],var[1],var[2],MSiGe(var[5]),vSiGeT_PH(var[5]),var[18])

tauPHALLLf(var)=tauPH_ALL(var[9],var[10],var[5],VSiGe(var[5]),var[1],var[2],MSiGe(var[5]),vSiGeL_PH(var[5]))
tauPHALLTf(var)=tauPH_ALL(var[9],var[10],var[5],VSiGe(var[5]),var[1],var[2],MSiGe(var[5]),vSiGeT_PH(var[5]))

tauPHeLf(var)=tauPH_e(var[14],md,MSiGe(var[5])/VSiGe(var[5]),(var[13][var[26],var[27]])[var[2]],var[1],var[2],
vSiGeL_PH(var[5]))
tauPHeTf(var)=tauPH_e(var[14],md,MSiGe(var[5])/VSiGe(var[5]),(var[13][var[26],var[27]])[var[2]],var[1],var[2],
vSiGeT_PH(var[5]))

tauPHBLf(var)=tauPH_B(vSiGeL_PH(var[5]),var[14],var[15])
tauPHBTf(var)=tauPH_B(vSiGeT_PH(var[5]),var[14],var[15])
#println("3")
tauPHTOTL=tau_phonon_B([tauPHNLf,tauPHULf,tauPHALLLf,tauPHeLf],vartauPH0)#,tauPH_eLf ,tauPHALLLf ,tauPHBLf
tauPHNL=tau_phonon_B([tauPHNLf],vartauPH0)
tauPHRL=tau_phonon_B([tauPHULf,tauPHALLLf,tauPHeLf],vartauPH0) #,tauPHALLLf,tauPH_eLf] ,tauPHBLf

tauPHTOTTx=tau_phonon_B([tauPHNTf,tauPHUTf,tauPHALLTf,tauPHeTf],vartauPH0) #tauPHALLTf,tauPH_eTf ,tauPHBTf
tauPHNTx=tau_phonon_B([tauPHNTf],vartauPH0)
tauPHRTx=tau_phonon_B([tauPHUTf,tauPHALLTf,tauPHeTf],vartauPH0) #,tauPHALLTf,tauPH_eTf ,tauPHBTf

tauPHTOTTy=tau_phonon_B([tauPHNTf,tauPHUTf,tauPHALLTf,tauPHeTf],vartauPH0) #,tauPHALLTf,tauPH_eTf tauPHBTf
tauPHNTy=tau_phonon_B([tauPHNTf],vartauPH0)
tauPHRTy=tau_phonon_B([tauPHUTf,tauPHALLTf,tauPHeTf],vartauPH0) #,tauPHALLTf,tauPH_eTf ,tauPHBTf
#println("4")

tauPHL=[tauPHTOTL,tauPHNL,tauPHRL]
tauPHTx=[tauPHTOTTx,tauPHNTx,tauPHRTx]
xmax=1.0

LoadError: UndefVarError: T_EF_Dict_Array not defined

In [5]:
@time klattice=thermalPropsiso(tauPHL,tauPHTx,tauPHTx,[vSiGeL_PH(xsp),vSiGeT_PH(xsp),vSiGeT_PH(xsp)])
xmax=3.0

LoadError: UndefVarError: thermalPropsiso not defined

In [6]:
SiGe_Min_1700_Bulk_EC_T=[309.4827586207,389.6551724138,450,479.3103448276,504.3103448276,589.6551724138,979.3103448276,
    1039.6551724138,1110.3448275862,1189.6551724138,1199.1379310345]
SiGe_Min_1700_Bulk_EC_E=[1252.3364485981,1113.707165109,1054.5171339564,1054.5171339564,912.7725856698,834.8909657321,
    526.4797507788,434.5794392523,478.1931464174,557.6323987539,501.5576323988]
SiGe_Min_730_Bulk_EC_T=[308.6206896552,360.3448275862,419.8275862069,509.2672413793,609.2432950192,957.662835249,
    1009.1475095786,1098.9463601533,1149.2337164751,1218.0795019157]
SiGe_Min_730_Bulk_EC_E=[771.0280373832,716.5109034268,626.1682242991,556.7670474213,455.0882658359,277.6912426445,
        273.3644859813,241.9955001731,231.1786085151,213.8715818622]    
SiGe_Min_230_Bulk_EC_T=[299.7365900383,398.5153256705,500.8860153257,598.4674329502,708.6206896552,868.4626436782,
        979.8132183908,1019.3247126437,1088.1704980843,1148.0363984674]
SiGe_Min_230_Bulk_EC_E=[314.4686742818,280.9363101419,249.5673243337,217.1166493596,184.6659743856,155.460366909,
        130.5815160955,131.6632052613,126.2547594323,131.6632052613]
SiGe_Min_22_Bulk_EC_T=[309.9137931034,348.2279693487,389.535440613,569.1331417625,690.0622605364,808.596743295,
    909.7701149425,989.3917624521,1079.1906130268]
SiGe_Min_22_Bulk_EC_E=[49.4548286604,54.8632744894,44.0463828314,38.6379370024,31.0661128418,25.6576670128,24.575977847,
    31.0661128418,45.1280719972]
xmax=1
SiGe_Min_1700_Bulk_SC_T=[310.6382978723,450.2127659574,500.4255319149,579.5744680851,779.5744680851,970.2127659574,
    1050.2127659575,1100.4255319149,1151.4893617021,1159.1489361702]
SiGe_Min_1700_Bulk_SC_S=[-105.0763607884,-144.5542357395,-159.9686724971,-169.9060174912,-220.5195144237,-245.9652786842,
    -276.149327764,-276.2263412087,-260.3537397207,-240.1200887613]
SiGe_Min_730_Bulk_SC_T=[310.6382978723,361.7021276596,419.5744680851,509.7872340426,610.2127659574,1009.3617021277,
    1219.5744680851]
SiGe_Min_730_Bulk_SC_S=[-165.1990601749,-180.0013053126,-210.1514162642,-220.7192272549,-250.3211069051,-330.687899752,
    -346.3477352826]
SiGe_Min_230_Bulk_SC_T=[298.7234042553,400.8510638298,499.5744680851,600.8510638298,710.6382978723,859.5744680851,
    978.7234042553,1039.1489361702,1090.2127659575,1159.1489361702]
SiGe_Min_230_Bulk_SC_S=[-249.8433624853,-284.9693251534,-305.3661401906,-320.245398773,-335.7512074142,-342.1146064482,
-342.9108471479,-330.7335856938,-309.3395118131,-251.1630335465]
SiGe_Min_22_Bulk_SC_T=[309.7872340426,349.7872340426,380.4255319149,410.2127659574,550.6382978723,690.2127659574,
824.6808510638,910.6382978723,979.5744680851]
SiGe_Min_22_Bulk_SC_S=[-479.9216812427,-489.7989818562,-519.9073228038,-524.8609842057,-534.8923117087,-544.922333899,
-540.2205978332,-480.8432319541,-372.9735021538]
xmax=1
SiGe_Min_1700_Bulk_KC_T=[314.5631067961,444.0129449838,499.2448759439,577.7777777778,769.3635382956,966.1272923409,
    1032.5782092772,1105.9331175836,1168.932038835,1188.7810140237]
SiGe_Min_1700_Bulk_KC_K=[4.6835063051,4.5239984377,4.4312520924,4.3076219172,4.4329985493,4.1911672804,4.0622865752,
4.2179332664,4.4666164491,4.3891585761]
SiGe_Min_22_Bulk_KC_T=[314.5631067961,378.4250269687,554.476806904,683.9266450917,814.2394822006,903.9913700108,
    973.0312837109]
SiGe_Min_22_Bulk_KC_K=[7.1197132017,6.4011605848,5.1350574713,4.6289978797,4.3608749024,4.4855931258,4.7291429528]
SiGe_Vin_162_Bulk_KCL_T=[440.3713548665536,497.3222430383224,574.0370374003198]
SiGe_Vin_162_Bulk_KCL_KL=[3.3457249070631976,3.2416356877323427,3.074349442379183]

SiGe_Su_162_Bulk_KCL_T=[51.5096343499,56.6540954115,60.7039902898,64.5349719314,68.5191928387,73.5104146347,78.545419078,
84.2006776918,89.8194507662,96.7152177211,105.1433773327,115.103929601,126.5968745259,140.3884084357,156.4785313306,
173.3348505538,190.191169777,207.0474890002,223.9038082233,240.7601274465,257.6164466697,274.4727658929,291.3290851161,
308.1854043393,325.0417235624,341.8980427856,358.7543620088,375.610681232,392.4670004552,409.3233196784,424.6472462449,
470.6190259445,487.4753451677,504.3316643908,521.187983614,538.0443028372,554.9006220604,571.7569412836,588.6132605068,
605.4695797299,622.3258989531,639.1822181763,656.0385373995,672.8948566227,689.7511758459,706.607495069,723.4638142922,
    740.3201335154,757.1764527386,774.0327719618,438.2642998028,452.4654832347]

SiGe_Su_162_Bulk_KCL_KL=[1.912952409,2.0367544267,2.1416796981,2.2300607933,2.316951567,2.4101428367,2.4960198089,
2.5805877859,2.6667920069,2.747450222,2.8331721816,2.9171373319,2.9981959963,3.0776786993,3.1519579117,3.2134462372,
3.2637548671,3.3065007618,3.3406974776,3.3686467164,3.3926501804,3.4110637965,3.4258604524,3.4370401479,3.443945254,
3.4469045852,3.4531520621,3.4482198435,3.4452605123,3.4409859228,3.4325828837,3.4025146176,3.3880467763,3.3719348622,
3.355822948,3.3383957755,3.3196533448,3.3002532849,3.2795379667,3.2575073902,3.2371208866,3.2157479392,3.1940461773,
3.1713579716,3.1483409514,3.1253239312,3.102306911,3.0792898907,3.0549576122,3.031940592,3.426443203,3.4189944134]

SiGe_Dis_163_Bulk_EC_T=[323.9316239316,333.0484330484,355.8404558405,400.2849002849,420.7977207977,459.5441595442,
506.2678062678,592.8774928775,923.3618233618,976.9230769231,1052.1367521368,1100,1171.7948717949,1237.8917378917,
1301.7094017094]
SiGe_Dis_163_Bulk_EC_E=[73051.8567141787,70171.3693787016,69094.6584181468,62330.1029133787,59820.8886457225,
56061.6010146897,51826.1650533879,45021.2384777646,28203.3180374162,27942.9501463909,25034.847614495,23576.1869994666,
22113.3307312919,21632.8230761221,22345.0026532046]

SiGe_Dis_162_Bulk_EC_T=[334.188034188,441.3105413105,493.7321937322,515.3846153846,549.5726495727,571.2250712251,
    631.6239316239,999.7150997151,1066.9515669516,1141.0256410256,1201.4245014245]
SiGe_Dis_162_Bulk_EC_E=[129575.649208891,107765.731182238,98298.134098831,92200.4911678551,92241.4303475224,
86851.457000711,78465.4898899606,45601.3521918239,48886.9016802372,52685.3268634132,55337.3230807481]

SiGe_Dis_68_Bulk_EC_T=[312.9046312215,345.9769133021,382.411609555,464.3824148131,558.8426751524,687.3302935492,
821.5449507192,919.5265198031,979.0333913526,1074.0899334736,1157.5745085267]
SiGe_Dis_68_Bulk_EC_E=[4928.7324041511,5086.1164396805,4873.7722800721,4417.6363921119,3850.5692775171,
2982.0808298762,2479.3727581675,2440.9309667491,2902.792845851,4527.732464084,6966.3728769178]


SiGe_Dis_42_Bulk_EC_T=[311.396011396,320.5128205128,343.3048433048,386.6096866097,494.8717948718,594.0170940171,
604.2735042735,738.7464387464,793.4472934473,819.6581196581,949.5726495727,973.5042735043,1001.9943019943,1045.2991452992,
1112.5356125356,1172.9344729345]
SiGe_Dis_42_Bulk_EC_E=[30555.8415579449,30768.8509940805,30455.2007232389,27362.653284335,25342.38776838,
    21749.1598295579,21696.6878102784,7862.4247635426,16113.8690660875,15660.0137196616,13456.377215578,
12853.0240347136,12724.7223702506,12817.5754746279,12690.8556278229,13182.2248732478]

SiGe_Dis_162_Bulk_KC_T=[323.4130104234,436.3766249262,465.5560008689,494.6582798193,568.7099409229,676.8421439083,
    861.4563988974,997.8960475554,1141.7663777569,1201.6652338456]
SiGe_Dis_162_Bulk_KC_K=[4.6738529713,4.5204119777,4.5006535608,4.430757327,4.3128769847,4.2031161293,3.9957878466,
4.065915243,4.2121420299,4.3719065138]

SiGe_SA_162_Bulk_KC_T=[398.7219168438,498.0855086789,598.5416158272,698.9878915666,797.1262440682,897.5381098765,
997.8565773003,1097.3037361109,1199.1374694305]
SiGe_SA_162_Bulk_KC_KL=[3.7593499332,3.62237734,3.494337284,3.3752157204,3.3497102368,3.2618033964,3.2586222334,
3.0458424552,2.6680986582]

figure()
plot(Ts,sigmas[:,1,1]/100,linestyle="-",color="b")
plot(Ts,sigmas[:,2,2]/100,linestyle="-",color="r")
plot(Ts,sigmas[:,3,3]/100,linestyle="-",color="g")
plot(Ts,sigmas[:,4,4]/100,linestyle="-",color="#ff66cc") 
plot(SiGe_Min_1700_Bulk_EC_T,SiGe_Min_1700_Bulk_EC_E, linestyle="None",marker="o",markersize=7,color="b")
plot(SiGe_Min_730_Bulk_EC_T,SiGe_Min_730_Bulk_EC_E, linestyle="None",marker="s",markersize=7,color="r")
plot(SiGe_Min_230_Bulk_EC_T,SiGe_Min_230_Bulk_EC_E, linestyle="None",marker="^",markersize=7,color="g")
plot(SiGe_Min_22_Bulk_EC_T,SiGe_Min_22_Bulk_EC_E, linestyle="None",marker="D",markersize=7,color="#ff66cc")
#plot(SiGe_Dis_42_Bulk_EC_T,SiGe_Dis_42_Bulk_EC_E/100, linestyle="None",marker="o",markersize=7,color="#80ff00")
#plot(SiGe_Dis_68_Bulk_EC_T,SiGe_Dis_68_Bulk_EC_E/100, linestyle="None",marker="s",markersize=7,color="#40ff00")
#plot(SiGe_Dis_162_Bulk_EC_T,SiGe_Dis_162_Bulk_EC_E/100, linestyle="None",marker="^",markersize=7,color="#00bfff")
#plot(SiGe_Dis_163_Bulk_EC_T,SiGe_Dis_163_Bulk_EC_E/100, linestyle="None",marker="D",markersize=7,color="#bf00ff")
#ylim(0,1500)
grid()

figure()
plot(Ts,seebecks[:,1,1]*1e6,linestyle="-",color="b")
plot(Ts,seebecks[:,2,2]*1e6,linestyle="-",color="r")
plot(Ts,seebecks[:,3,3]*1e6,linestyle="-",color="g")
plot(Ts,seebecks[:,4,4]*1e6,linestyle="-",color="#ff66cc") 
plot(SiGe_Min_1700_Bulk_SC_T,SiGe_Min_1700_Bulk_SC_S, linestyle="None",marker="o",markersize=7,color="b")
plot(SiGe_Min_730_Bulk_SC_T,SiGe_Min_730_Bulk_SC_S, linestyle="None",marker="s",markersize=7,color="r")
plot(SiGe_Min_230_Bulk_SC_T,SiGe_Min_230_Bulk_SC_S, linestyle="None",marker="^",markersize=7,color="g")
plot(SiGe_Min_22_Bulk_SC_T,SiGe_Min_22_Bulk_SC_S, linestyle="None",marker="D",markersize=7,color="#ff66cc")
ylim(-600,0)
grid()

figure()
plot(Ts,klattice[:,1,1]+kes[:,1,1],linestyle="-",color="b")
#plot(Ts,klattice[:,2,2]+kes[:,2,2],linestyle="-",color="r")
#plot(Ts,klattice[:,3,3]+kes[:,3,3],linestyle="-",color="g")
plot(Ts,klattice[:,4,4]+kes[:,4,4],linestyle="-",color="#ff66cc") 
plot(SiGe_Min_1700_Bulk_KC_T,SiGe_Min_1700_Bulk_KC_K, linestyle="None",marker="o",markersize=7,color="b")
#plot(SiGe_Min_1700_Bulk_KC_T,SiGe_Min_1700_Bulk_KC_K,linestyle="None",marker="o",markersize=7,color="b")
#plot(SiGe_Min_730_Bulk_KC_T,SiGe_Min_730_Bulk_SC_S, linestyle="None",marker="s",markersize=7,color="r")
#plot(SiGe_Min_230_Bulk_SC_T,SiGe_Min_230_Bulk_SC_S, linestyle="None",marker="^",markersize=7,color="g")
plot(SiGe_Min_22_Bulk_KC_T,SiGe_Min_22_Bulk_KC_K, linestyle="None",marker="D",markersize=7,color="#ff66cc")
plot(SiGe_Dis_162_Bulk_KC_T,SiGe_Dis_162_Bulk_KC_K, linestyle="None",marker="D",markersize=7,color="k")
ylim(3,8)
grid()

figure()
plot(Ts,klattice[:,1,1],linestyle="-.",color="b",label="kl")
#plot(Ts,klattice[:,2,2],linestyle="-.",color="r")
#plot(Ts,klattice[:,3,3],linestyle="-.",color="g")
plot(Ts,klattice[:,4,4],linestyle="-.",color="#ff66cc") 
plot(SiGe_Vin_162_Bulk_KCL_T,SiGe_Vin_162_Bulk_KCL_KL, linestyle="None",marker="s",markersize=4,color="c")
plot(SiGe_Su_162_Bulk_KCL_T,SiGe_Su_162_Bulk_KCL_KL, linestyle="None",marker="s",markersize=4,color="c")
plot(SiGe_SA_162_Bulk_KC_T,SiGe_SA_162_Bulk_KC_KL,linestyle="None",marker="o",markersize=7,color="b")
plot(Ts,kes[:,1,1],linestyle="--",color="b",label="ke")
plot(Ts,kees[:,1,1],linestyle="--",color="k",label="kee")
plot(Ts,kehs[:,1,1],linestyle="--",color="y",label="keh")
#plot(Ts,kes[:,2,2],linestyle="--",color="r")
#plot(Ts,kes[:,3,3],linestyle="--",color="g")
plot(Ts,kes[:,4,4],linestyle="--",color="#ff66cc") 
plot(Ts,kehs[:,4,4],linestyle="--",color="#ff66cc",label="keh")
plot(Ts,kbis[:,1,1],linestyle=":",color="b",label="kbis")
#plot(Ts,kbis[:,2,2],linestyle=":",color="r")
#plot(Ts,kbis[:,3,3],linestyle=":",color="g")
plot(Ts,kbis[:,4,4],linestyle=":",color="#ff66cc") 
#ylim()
legend(loc=1)
grid()
#plot(SiGe_Vin_162_Bulk_KC_T,SiGe_Vin_162_Bulk_KC_K, linestyle="None",marker="s",markersize=4,color="c")
#plot(Ts,kl_A,Linestyle="-",Linewidth="2",color="b")




LoadError: UndefVarError: figure not defined